In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

from src.trust_region_sqp import TrustRegionSQPFilter
import numpy as np
import casadi as ca


In [ ]:
## Constants

constants = dict()
constants["gamma_0"] = 0.2
constants["gamma_1"] = 0.7
constants["gamma_2"] = 1.2 #1.5 Eq
constants["eta_1"] = 0.1
constants["eta_2"] = 0.4
constants["mu"] = 0.01
constants["gamma_vartheta"] = 1E-8 #1E-4 
constants["kappa_vartheta"] = 1E-4
constants["kappa_radius"] = 0.8
constants["kappa_mu"] = 10
constants["kappa_tmd"] = 0.01

constants["init_radius"] = 1.
constants["stopping_radius"] = 1E-3
constants["L_threshold"] = 1.2

dataset = np.array([[0.0, 0.0], [1.0, 0.0], [0.0, 1.0], [-1.0, 0.0], [1.0, 1.0]], dtype=np.float32).T + np.array([[2.1], [-2.12]], dtype=np.float32) 

tr_list = []

for c1, c2, name in [(0.0,np.nan,'PQ-E1'), (1.0,np.nan,'PQ-E2'), (-1.0,np.nan,'PQ-E3'), 
                     (np.nan,0.0,'PQ-I1'), (np.nan,1,'PQ-I2'), (np.nan,-1.0,'PQ-I3'), 
                     (0.0,0.0,'PQ-IE1'), (1.0,1.0,'PQ-IE2'), (-1.0,-1.0,'PQ-IE3'),
                     (0.0,0.0,'PQ-IE-N1'), (1.0,1.0,'PQ-IE-N2'), (-1.0,-1.0,'PQ-IE-N3'),
                     (0.0,np.nan,'RB-E1'), (1.0,np.nan,'RB-E2'), (-1.0,np.nan,'RB-E3'), 
                     (np.nan,0.0,'RB-I1'), (np.nan,1,'RB-I2'), (np.nan,-1.0,'RB-I3'), 
                     (0.0,0.0,'RB-IE1'), (1.0,1.0,'RB-IE2'), (-1.0,-1.0,'RB-IE3'),
                     (0.0,0.0,'RB-IE-N1'), (1.0,1.0,'RB-IE-N2'), (-1.0,-1.0,'RB-IE-N3')]:
# for c1, c2, name in [(np.nan,-1,'PQ-I3')]:
    tr_dict = dict()
    xeq_plot_list = []
    yeq_plot_list = []
    xineq_plot_list = []
    yineq_plot_list = []
#     def cf(x:np.ndarray) -> np.ndarray: # Rosenbrock function: OF
#         return 1*(x[1]-x[0]**2)**2+((x[0]-1)**2)/100

    if 'N' in name:
        
        if 'RB' in name:
            def cf(x:np.ndarray) -> np.ndarray: # Rosenbrock function: OF
                return 1*(x[1]-x[0]**2)**2+((x[0]-1)**2)/100
        elif 'PQ' in name:
            def cf(x:np.ndarray) -> np.ndarray:
                return 10*(x[0]**2)*(1 + 0.75*np.cos(70*x[0])/12) + np.cos(100*x[0])**2/24 + 2*(x[1]**2)*(1 + 0.75*np.cos(70*x[1])/12) + np.cos(100*x[1])**2/24 + 4*x[0]*x[1]

    #     def cf(x:np.ndarray) -> np.ndarray:
    #         return 10*(x[0]**2) + 2*(x[1]**2) + 4*x[0]*x[1]

        def eq_constraint(x:np.ndarray) -> np.ndarray: # equality constraints
            return x[1]**2 - x[0] + c1

        def ineq_constraint(x:np.ndarray) -> np.ndarray: # inequality constraints
            return x[1]**2 + x[0] + c2
        
        xmin = -3 
        xmax = 3
        ymin = -3
        ymax = 3
        N = 100

        # Equality
        yeq_plot = []
        xeq_plot = []
        for y in list(np.linspace(-3,3,N)):
            x = y**2 + c1 # change this
            if x <= xmax and x >= xmin:
                xeq_plot.append(x)
                yeq_plot.append(y)
        yeq_plot = np.array(yeq_plot)
        xeq_plot = np.array(xeq_plot)

        yeq_plot_list.append(yeq_plot)
        xeq_plot_list.append(xeq_plot)

        # Inequality
        yineq_plot = []
        xineq_plot = []
        for y in list(np.linspace(-3,3,N)):
            x = -y**2 - c2 # change this
            if x <= xmax and x >= xmin:
                xineq_plot.append(x)
                yineq_plot.append(y)
        yineq_plot = np.array(yineq_plot)
        xineq_plot = np.array(xineq_plot)

        yineq_plot_list.append(yineq_plot)
        xineq_plot_list.append(xineq_plot)
    
    else:
        if 'RB' in name:
            def cf(x:np.ndarray) -> np.ndarray: # Rosenbrock function: OF
                return 1*(x[1]-x[0]**2)**2+((x[0]-1)**2)/100
        elif 'PQ' in name:
            def cf(x:np.ndarray) -> np.ndarray:
                return 10*(x[0]**2)*(1 + 0.75*np.cos(70*x[0])/12) + np.cos(100*x[0])**2/24 + 2*(x[1]**2)*(1 + 0.75*np.cos(70*x[1])/12) + np.cos(100*x[1])**2/24 + 4*x[0]*x[1]
        
    #     def cf(x:np.ndarray) -> np.ndarray:
    #         return 10*(x[0]**2) + 2*(x[1]**2) + 4*x[0]*x[1]

        def eq_constraint(x:np.ndarray) -> np.ndarray: # equality constraints
            return x[1] - x[0] + c1

        def ineq_constraint(x:np.ndarray) -> np.ndarray: # inequality constraints
            return x[1] + x[0] + c2

        xmin = -3 
        xmax = 3
        ymin = -3
        ymax = 3
        N = 100

        # Equality
        yeq_plot = []
        xeq_plot = []
        for y in list(np.linspace(-3,3,N)):
            x = y + c1 # change this
            if x <= xmax and x >= xmin:
                xeq_plot.append(x)
                yeq_plot.append(y)
        yeq_plot = np.array(yeq_plot)
        xeq_plot = np.array(xeq_plot)

        yeq_plot_list.append(yeq_plot)
        xeq_plot_list.append(xeq_plot)

        # Inequality
        yineq_plot = []
        xineq_plot = []
        for y in list(np.linspace(-3,3,N)):
            x = - y - c2 # change this
            if x <= xmax and x >= xmin:
                xineq_plot.append(x)
                yineq_plot.append(y)
        yineq_plot = np.array(yineq_plot)
        xineq_plot = np.array(xineq_plot)

        yineq_plot_list.append(yineq_plot)
        xineq_plot_list.append(xineq_plot)
    
    if np.isnan(c1):
        eqcs = []
    else:
        eqcs = [eq_constraint]
        
    if np.isnan(c2):
        ineqcs = []
    else:
        ineqcs = [ineq_constraint]
    
    tr = TrustRegionSQPFilter(constants = constants, 
                          dataset = dataset, 
                          cf = cf, 
                          eqcs = eqcs, 
                          ineqcs = ineqcs)

    tr.run(max_iter=35)
    
    tr_dict['name'] = name
    tr_dict['tr'] = tr
    tr_dict['cf'] = cf
    tr_dict['plot'] = [xeq_plot_list, 
                       yeq_plot_list,
                       xineq_plot_list,
                       yineq_plot_list]
    tr_list.append(tr_dict)

# Make table of results

In [ ]:
for tr in tr_list:
    print(tr['name'], '&',
          tr['tr'].iterates[-1]['iteration_no'],  '&',
          tr['tr'].iterates[-1]['number_of_function_calls'],  '&',
          tr['tr'].termination_status, '&',
          "{:.2e}".format(tr['tr'].iterates[-1]['y_curr'][0]), '&',
          "{:.2e}".format(tr['tr'].iterates[-1]['y_curr'][1]), '&',
          "{:.2e}".format(tr['tr'].iterates[-1]['v'][0]),  '&',
          "{:.2e}".format(tr['tr'].iterates[-1]['fY'][0]), '\\\\')

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib import ticker, cm

ind = 15
tr = tr_list[ind]
xeq_plot = tr['plot'][0][0] #xeq_plot_list[ind]
yeq_plot = tr['plot'][1][0] #yeq_plot_list[ind]
xineq_plot = tr['plot'][2][0] #xineq_plot_list[ind]
yineq_plot = tr['plot'][3][0] #yineq_plot_list[ind]

X, Y = np.meshgrid(np.linspace(-3, 3, 100),
                    np.linspace(-3, 3, 100))
# levels = [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5, 10, 50]
levels = [10**i for i in np.arange(-1,2.5,0.2)]

fsize = 15
tsize = 18
tdir = 'in'
major = 5.0
minor = 3.0

style = 'default'

plt.style.use(style)
plt.rcParams['text.usetex'] = True
plt.rcParams['font.size'] = fsize
plt.rcParams['legend.fontsize'] = tsize
plt.rcParams['xtick.direction'] = tdir
plt.rcParams['ytick.direction'] = tdir
plt.rcParams['xtick.major.size'] = major
plt.rcParams['xtick.minor.size'] = minor
plt.rcParams['ytick.major.size'] = major
plt.rcParams['ytick.minor.size'] = minor

for i in range(len(tr['tr'].iterates)):
    if i % 3 == 0:
        center = tr['tr'].iterates[i]['y_curr']
        radius = tr['tr'].iterates[i]['radius']
        intx = X - center[0]
        inty = Y - center[1]
        dist = np.sqrt(intx**2 + inty**2)
        intindices = dist <= radius

        func = tr['cf']([X, Y])

        func_map = tr['tr'].iterates[i]['models'].m_cf.model.model_polynomial.feval.map(X.shape[0]*X.shape[1])
        x_dm = ca.DM(X).reshape((1, X.shape[0]*X.shape[1]))
        y_dm = ca.DM(Y).reshape((1, X.shape[0]*X.shape[1]))
        func_dm = func_map(ca.vertcat(x_dm, y_dm))
        func_dm = func_dm.reshape((X.shape[0], X.shape[1])).full()
        func[intindices] = func_dm[intindices]

        circle1 = plt.Circle(center, radius, color='black', fill=False)

        fig, ax = plt.subplots(1)
        ax.add_patch(circle1)
        CS = ax.contour(X, Y, func, levels, norm = LogNorm(), cmap=cm.PuBu_r)
        ax.clabel(CS, fontsize=9, inline=True)
        x = tr['tr'].iterates[i]['Y'][0,:]
        y = tr['tr'].iterates[i]['Y'][1,:]
        plt.scatter(x, y, color='black', label=f"iteration {i}")
        plt.scatter(x[0], y[0], color='red', label=f'Best point')

        #Eq constraint    
        plt.plot(xeq_plot, yeq_plot, 'b-', label=f"eq constraint")

        #Ineq constraint
        plt.plot(xineq_plot, yineq_plot, 'r--', label=f"ineq constraint")

        plt.legend()
    
    

In [ ]:
# Iteration vs OF
# print(tr_list)
for inds, name in [([0,1,2], 'PQ-E'), 
                   ([3,4,5], 'PQ-I'), 
                   ([6,7,8], 'PQ-IE'), 
                   ([9,10,11], 'PQ-IE-N'), 
                   ([12,13,14], 'RB-E'), 
                   ([15,16,17], 'RB-I'), 
                   ([18,19,20], 'RB-IE'), 
                   ([21,22,23], 'RB-IE-N')]:
    
    _tr_list = [tr_list[i] for i in inds]
    
    fig, ax = plt.subplots(1)
    for j, tr in enumerate(_tr_list):
        OF_list = [i['fY'][0] for i in tr['tr'].iterates]
        n_iter = [i['number_of_function_calls'] for i in tr['tr'].iterates] 
        ax.plot(n_iter, OF_list, label=f"{tr['name']}", linestyle='dashed')
    try:
        ax.set_yscale("log")
    except:
        ax.set_yscale("linear")
    ax.set_xlabel(f"Number of function evaluations")
    ax.set_ylabel(f"$f$")
    ax.legend()
    plt.savefig(f"./OF_{name}.png")

    fig, ax = plt.subplots(1)
    for j, tr in enumerate(_tr_list):
        v_list = [i['v'][0] for i in tr['tr'].iterates]
        n_iter = [i['number_of_function_calls'] for i in tr['tr'].iterates] 
        ax.plot(n_iter, v_list, label=f"{tr['name']}", linestyle='dashed')
    try:
        ax.set_yscale("log")
    except:
        ax.set_yscale("linear")

    ax.set_xlabel(f"Number of function evaluations")
    ax.set_ylabel(f"$\\vartheta$")
    ax.legend()
    plt.savefig(f"./VL_{name}.png")